In [147]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
from datetime import datetime
from shapely.geometry import shape, GeometryCollection, Point

In [48]:
df = pd.read_csv("../data_raw/climate_tweets.csv")

In [49]:
df = df.dropna()
df_uk = df[df["lat"].between(49.96514, 58.6350001085) & df["lng"].between(-7.57216793459, 1.68153079591)]

In [60]:
with open("../data_raw/countries.geojson", "r") as f:
    country_borders = json.load(f)

In [73]:
uk = [geo for geo in country_borders.get("features") if geo.get("properties").get("ADMIN") == "United Kingdom"][0].get("geometry")
uk_polygon = shape(uk)

In [82]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)
df_uk["UK"] = df_uk.parallel_apply(lambda row: uk_polygon.contains(Point(row["lng"], row["lat"])), axis=1)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [90]:
df_uk = df_uk.query('UK == True')
df_uk = df_uk.drop(["UK"], axis=1)

In [91]:
df_uk.to_csv("../data_clean/climate_tweets_uk.csv")

In [116]:
df_uk["date"] = df_uk["created_at"].apply(lambda x: datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))

In [157]:
month_group = df_uk["date"].dt.to_period("M")
month_series = df_uk.groupby(["topic", "aggressiveness", month_group]).size().reset_index(name="counts")
month_series["date"] = month_series["date"].astype(str)
month_series["date"] = pd.to_datetime(month_series['date'])

In [159]:
fig = go.Figure()
buttons = []
i = 0

topics = month_series["topic"].unique()
stances = month_series["aggressiveness"].unique()

for topic in topics:
    df_current_topic = month_series[month_series["topic"] == topic]

    for stance in stances:
        df_current_stance = df_current_topic[df_current_topic["aggressiveness"] == stance]
        fig.add_trace(
            go.Line(
                x=df_current_stance["date"].to_list(),
                y=df_current_stance["counts"].to_list(),
                name=stance,
                visible=(i == 0)
            )
        )

    args = [False] * len(topics) * len(stances)
    args[i*len(stances):(i*len(stances))+len(stances)] = [True] * len(stances)
    
    button = dict(label = topic,
                  method = "update",
                  args=[{"visible": args}])
    
    buttons.append(button)
    
    i += 1

fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            buttons=buttons
        )
    ],
    barmode="stack"
)

# Set title
fig.update_layout(title_text="Tweets")

fig.show()

/Users/luisdibdin/miniconda3/envs/hackathon/lib/python3.11/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [163]:
import plotly.offline as py

py.plot(fig, filename="twitter_timeseries.html")

'twitter_timeseries.html'

In [119]:
disasters = pd.read_csv("../data_raw/disasters.csv")

In [152]:
import plotly.offline as py

In [161]:
py.plot(fig, filename="test.html")

'test.html'